In [1]:
# plotly standard imports
import plotly.graph_objs as go
import chart_studio.plotly as py

# Cufflinks wrapper on plotly
import cufflinks

# Data science imports
import pandas as pd
import numpy as np

# Options for pandas
pd.options.display.max_columns = 30

# Display all cell outputs
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all'

from plotly.offline import iplot, init_notebook_mode
cufflinks.go_offline(connected=True)
init_notebook_mode(connected=True)

# Set global theme
cufflinks.set_config_file(world_readable=True, theme='pearl')

Base dataset of Wiki40b contains Wikidata ids, but not contain urls. Will load urls and add them to dataset

In [2]:
from datasets import load_dataset

wiki40b_snippets = load_dataset('wiki_snippets', name='wiki40b_en_100_0', cache_dir='./datasets')

Downloading:   0%|          | 0.00/2.11k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/580 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/2.03k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/639 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.38k [00:00<?, ?B/s]

...

Downloading:   0%|          | 0.00/527M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/522M [00:00<?, ?B/s]

Dataset wiki40b downloaded and prepared to /root/.cache/huggingface/datasets/wiki40b/en/1.1.0/3d99a1f96eba780a027018e9a18c1b9a6f5809625e78804a915495f6193dfa5e. Subsequent calls will reuse this data.


0 examples [00:00, ? examples/s]

Dataset wiki_snippets downloaded and prepared to ./datasets/wiki_snippets/wiki40b_en_100_0/1.0.0/d152a0e6a420c02b9b26e7f75f45fb54c818cae1d83e8f164f0b1a13ac7998ae. Subsequent calls will reuse this data.


In [3]:
wiki40b_snippets['train'][3]

{'_id': '{"datasets_id": 1, "wiki_id": "Q3588472", "sp": 8, "sc": 209, "ep": 8, "ec": 312}',
 'datasets_id': 1,
 'wiki_id': 'Q3588472',
 'start_paragraph': 8,
 'start_character': 209,
 'end_paragraph': 8,
 'end_character': 312,
 'article_title': 'Émile Dubonnet',
 'section_title': 'Biography',
 'passage_text': 'also helped form the first professional baseball league in France, the French Baseball Union, in 1912.'}

In [9]:
import requests

def get_page_url(wikidata_id, lang = 'enwiki'):

    url = f'https://www.wikidata.org/w/api.php?action=wbgetentities&ids={wikidata_id}&format=json&props=sitelinks/urls&sitefilter={lang}'

    response = requests.get(url).json()
    
    entities = response.get('entities')
    if not entities:
        return None

    entity = entities.get(wikidata_id)
    if not entity:
        return None

    sitelinks = entity.get('sitelinks')
    if not sitelinks:
        return None

    sitelink = sitelinks.get(lang)
    if not sitelink:
        return None

    wiki_url = sitelink.get('url')
    if not wiki_url:
        return wiki_url
    
    return requests.utils.unquote(wiki_url)

In [20]:
from tqdm.notebook import tqdm

train_dataset = wiki40b_snippets['train']

# def add_url(passage):
#     print('get page', passage['wiki_id'])
#     return {'page_url': get_page_url(passage['wiki_id'])}

# train_dataset = train_dataset.map(add_url, num_proc=12)
   
# print(train_dataset[0])

In [21]:
train_dataset.save_to_disk('./datasets/wiki_snippets_wiki40b_en_train_processed')